In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.linear_model import SGDRegressor


In [82]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt 
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [83]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [84]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

## PREPROCESSING

In [85]:
df_train.columns[df_train.isnull().any()]

Index(['LotFrontage', 'Alley', 'MasVnrType', 'MasVnrArea', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Electrical', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence',
       'MiscFeature'],
      dtype='object')

In [86]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [87]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [88]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [89]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [90]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [91]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [92]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [93]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [94]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [95]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [96]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [97]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int" or df_train[cols].dtype=="float":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
            elif for_numerical_cols=="mode":
                mode_=df_train[cols].mode()[0]
                df_train[cols].fillna(mode_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [98]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [99]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [100]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [101]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

Deleting Outliers.

Here we are deleting outliers using Inter Quartile Range Strategy. Since for a column ouliers are in a specifc row and for different columns the oulier contained rows are different and if we delete those specific row then that row in output label should also be deleted thats why we are first adding label into our features after that we will separate the label.

In [102]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here

In [103]:
for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)] 
    df_train = df_train[(df_train[col] >=l)] 



[ -55.0 , 145.0 ]
[ 30.0 , 110.0 ]
[ 2440.0 , 16936.0 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1909.0 , 2061.0 ]
[ -255.75 , 426.25 ]
[ -1060.5 , 1767.5 ]
[ 0.0 , 0.0 ]
[ -579.875 , 1725.125 ]
[ 199.0 , 2037.0 ]
[ -1092.375 , 1820.625 ]
[ 0.0 , 0.0 ]
[ -1.5 , 2.5 ]
[ 0.0 , 0.0 ]
[ -0.5 , 3.5 ]
[ -1.5 , 2.5 ]
[ 0.5 , 4.5 ]
[ 1.0 , 1.0 ]
[ 2.0 , 10.0 ]
[ -1.5 , 2.5 ]
[ 1903.0 , 2063.0 ]
[ -19.0 , 933.0 ]
[ -239.625 , 399.375 ]
[ -97.5 , 162.5 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.5 , 12.5 ]
[ 2004.0 , 2012.0 ]


In [104]:
output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

In [105]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
6,20,RL,75.0,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,57,0,0,0,0,0,8,2007,WD,Normal
10,20,RL,70.0,11200,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,RL,63.0,8500,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,60,0,0,0,0,0,11,2007,WD,Normal
1448,50,RL,70.0,11767,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,5,2007,WD,Normal
1451,20,RL,78.0,9262,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,5,2009,New,Partial
1454,20,FV,62.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,113,0,0,0,0,0,10,2009,WD,Normal


Here we are encoding our categorical columns using 'one_hot_encoding' as a strategy. We will be experimenting other strategies later.

In [106]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [107]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [108]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,0,0,0,1,0,0,0,0,1,0
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,0,0,0,1,0,0,0,0,1,0
6,20,75.0,10084,8,5,2005,186.0,1369,0,317,...,0,0,0,1,0,0,0,0,1,0
10,20,70.0,11200,5,5,1965,0.0,906,0,134,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,63.0,8500,7,5,2004,106.0,0,0,1422,...,0,0,0,1,0,0,0,0,1,0
1448,50,70.0,11767,4,7,2000,0.0,0,0,560,...,0,0,0,1,0,0,0,0,1,0
1451,20,78.0,9262,8,5,2009,194.0,0,0,1573,...,0,1,0,0,0,0,0,0,0,1
1454,20,62.0,7500,7,5,2005,0.0,410,0,811,...,0,0,0,1,0,0,0,0,1,0


In [125]:
df_train.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
count,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.0,576.000000,...,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000,576.000000
mean,48.315972,68.781250,9167.371528,6.263889,5.366319,1989.269097,77.281250,441.932292,0.0,614.366319,...,0.001736,0.119792,0.003472,0.835069,0.060764,0.001736,0.001736,0.012153,0.802083,0.121528
std,31.355368,13.663944,2609.625982,1.316047,0.704898,19.823626,108.909316,416.412209,0.0,428.094824,...,0.041667,0.325000,0.058874,0.371440,0.239104,0.041667,0.041667,0.109663,0.398775,0.327024
min,20.000000,30.000000,2887.000000,2.000000,4.000000,1950.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,60.000000,7717.250000,5.000000,5.000000,1972.000000,0.000000,0.000000,0.0,280.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,50.000000,69.000000,9017.500000,6.000000,5.000000,1999.000000,0.000000,428.000000,0.0,517.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,60.000000,75.000000,10800.000000,7.000000,6.000000,2005.000000,147.250000,743.000000,0.0,879.250000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,120.000000,110.000000,16770.000000,10.000000,7.000000,2010.000000,423.000000,1619.000000,0.0,1709.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Taking features in X and lables in Y

In [110]:
X=df_train
Y=output_col

The below dictionary will save the final accuracies of each model.

In [111]:
avg_accuracies={}
highest_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    avg_accuracies[al]=0
    highest_accuracies[al]=0

In [112]:
def algo_testing(algo:str,test_size:float,X,Y,evaluation_metric:str,random_state,shuffle):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=random_state,shuffle=shuffle)
   
    model.fit(X_train,Y_train)
    predicted=model.predict(X_test)
    
    r2=evaluation_metric(Y_test, predicted)
    print(f"Testing r2_score of {model} is ",r2)
    print(f"Training r2_score of {model} is ", model.score(X_train,Y_train))
    if highest_accuracies[algo]<r2:
        
        highest_accuracies[algo]=r2
    avg_accuracies[algo]+=r2



In [113]:
for i in range(50):
    algo_testing(algo="LinearRegression()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["LinearRegression()"]/=50

Testing r2_score of LinearRegression() is  0.8754005113168507
Training r2_score of LinearRegression() is  0.9631121430821149
Testing r2_score of LinearRegression() is  0.8648294435372681
Training r2_score of LinearRegression() is  0.9645287871896047
Testing r2_score of LinearRegression() is  0.9082023881886754
Training r2_score of LinearRegression() is  0.9597103339789386
Testing r2_score of LinearRegression() is  0.9024419147903552
Training r2_score of LinearRegression() is  0.9610644348614501
Testing r2_score of LinearRegression() is  0.8274920405842754
Training r2_score of LinearRegression() is  0.9632942505295693
Testing r2_score of LinearRegression() is  0.8990633381101949
Training r2_score of LinearRegression() is  0.9598069677127345
Testing r2_score of LinearRegression() is  0.9082867317076274
Training r2_score of LinearRegression() is  0.9567901479971159
Testing r2_score of LinearRegression() is  0.8749301124971917
Training r2_score of LinearRegression() is  0.9613259161358902


In [114]:
for i in range(50):

    algo_testing(algo="RandomForestRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["RandomForestRegressor()"]/=50

Testing r2_score of RandomForestRegressor() is  0.8512330441399092
Training r2_score of RandomForestRegressor() is  0.9838090774201033
Testing r2_score of RandomForestRegressor() is  0.8728253599764912
Training r2_score of RandomForestRegressor() is  0.9823983257539344
Testing r2_score of RandomForestRegressor() is  0.8477806942896132
Training r2_score of RandomForestRegressor() is  0.9818382895860379
Testing r2_score of RandomForestRegressor() is  0.88493501521825
Training r2_score of RandomForestRegressor() is  0.9822534021385207
Testing r2_score of RandomForestRegressor() is  0.8930034883092894
Training r2_score of RandomForestRegressor() is  0.9795536099556725
Testing r2_score of RandomForestRegressor() is  0.9145958102361684
Training r2_score of RandomForestRegressor() is  0.9808447850084829
Testing r2_score of RandomForestRegressor() is  0.8848493974338937
Training r2_score of RandomForestRegressor() is  0.9788314483537566
Testing r2_score of RandomForestRegressor() is  0.8984860

In [115]:
for i in range(50):

    algo_testing(algo="SVR()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SVR()"]/=50

Testing r2_score of SVR() is  -0.021642656391917647
Training r2_score of SVR() is  -0.006377324667438122
Testing r2_score of SVR() is  -0.036098032836018534
Training r2_score of SVR() is  -0.008842055283748707
Testing r2_score of SVR() is  -0.029728166109321164
Training r2_score of SVR() is  -0.005362574862724845
Testing r2_score of SVR() is  -0.0029853900341239026
Training r2_score of SVR() is  -0.009109781389398552
Testing r2_score of SVR() is  -0.04837289540994094
Training r2_score of SVR() is  -0.010403994416371853
Testing r2_score of SVR() is  -0.010965047802799166
Training r2_score of SVR() is  -0.008314679820426107
Testing r2_score of SVR() is  -0.0036736333289935086
Training r2_score of SVR() is  -0.003908171206683075
Testing r2_score of SVR() is  -0.004265132724800225
Training r2_score of SVR() is  -0.00847773484908565
Testing r2_score of SVR() is  -0.002079011980636425
Training r2_score of SVR() is  -0.011206493822258023
Testing r2_score of SVR() is  -0.027569891980751215
Tra

In [116]:
for i in range(50):

    algo_testing(algo="KNeighborsRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["KNeighborsRegressor()"]/=50

Testing r2_score of KNeighborsRegressor() is  0.6596777649833276
Training r2_score of KNeighborsRegressor() is  0.792902013118727
Testing r2_score of KNeighborsRegressor() is  0.6774454194593011
Training r2_score of KNeighborsRegressor() is  0.7804770921653066
Testing r2_score of KNeighborsRegressor() is  0.5649866108924444
Training r2_score of KNeighborsRegressor() is  0.7947142231377626
Testing r2_score of KNeighborsRegressor() is  0.6200478504165546
Training r2_score of KNeighborsRegressor() is  0.7764757835907413
Testing r2_score of KNeighborsRegressor() is  0.6484440266001789
Training r2_score of KNeighborsRegressor() is  0.7814896813849557
Testing r2_score of KNeighborsRegressor() is  0.6764466334033706
Training r2_score of KNeighborsRegressor() is  0.7876027234391353
Testing r2_score of KNeighborsRegressor() is  0.7013809410842169
Training r2_score of KNeighborsRegressor() is  0.7740942067131575
Testing r2_score of KNeighborsRegressor() is  0.6506887350792474
Training r2_score o

In [117]:
for i in range(50):

    algo_testing(algo="SGDRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SGDRegressor()"]/=50

Testing r2_score of SGDRegressor() is  -7.148844852420167e+22
Training r2_score of SGDRegressor() is  -5.90766788996048e+22
Testing r2_score of SGDRegressor() is  -9.75731556362714e+22
Training r2_score of SGDRegressor() is  -9.895985351079984e+22
Testing r2_score of SGDRegressor() is  -1.538933936163247e+22
Training r2_score of SGDRegressor() is  -2.0861380380055025e+22
Testing r2_score of SGDRegressor() is  -8.752666375378366e+22
Training r2_score of SGDRegressor() is  -7.236230629204506e+22
Testing r2_score of SGDRegressor() is  -1.96388344610431e+24
Training r2_score of SGDRegressor() is  -1.8438945044425824e+24
Testing r2_score of SGDRegressor() is  -9.780122221449394e+23
Training r2_score of SGDRegressor() is  -8.4579104238832e+23
Testing r2_score of SGDRegressor() is  -1.8548136002409467e+23
Training r2_score of SGDRegressor() is  -1.4507295424933484e+23
Testing r2_score of SGDRegressor() is  -1.0375613689262353e+22
Training r2_score of SGDRegressor() is  -6.926240832380244e+21


In [118]:
for i in range(50):

    algo_testing(algo="DecisionTreeRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["DecisionTreeRegressor()"]/=50

Testing r2_score of DecisionTreeRegressor() is  0.7020326985072098
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.5912804229346615
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.5490492961954392
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.8290469023364402
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.733080474789148
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7562455618066131
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.593109998194584
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.844324285786225
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor()

In [119]:
for i in range(50):

    algo_testing(algo="ElasticNet()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["ElasticNet()"]/=50

Testing r2_score of ElasticNet() is  0.8947442179466402
Training r2_score of ElasticNet() is  0.8995377735169581
Testing r2_score of ElasticNet() is  0.8911009270539095
Training r2_score of ElasticNet() is  0.9002269352796785
Testing r2_score of ElasticNet() is  0.8747881451267057
Training r2_score of ElasticNet() is  0.90180036421522
Testing r2_score of ElasticNet() is  0.8998942503153217
Training r2_score of ElasticNet() is  0.899046032715696
Testing r2_score of ElasticNet() is  0.913439213697536
Training r2_score of ElasticNet() is  0.895709403047663
Testing r2_score of ElasticNet() is  0.8809649171411558
Training r2_score of ElasticNet() is  0.9015481770672595
Testing r2_score of ElasticNet() is  0.8897388362467412
Training r2_score of ElasticNet() is  0.9013456938249838
Testing r2_score of ElasticNet() is  0.858475916691766
Training r2_score of ElasticNet() is  0.9045376048604603
Testing r2_score of ElasticNet() is  0.8841629630100686
Training r2_score of ElasticNet() is  0.899916

In [120]:
for i in range(50):

    algo_testing(algo="Lasso()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Lasso()"]/=50

C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.444e+10, tolerance: 1.878e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.8714024077252679
Training r2_score of Lasso() is  0.9631308004302375


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.043e+10, tolerance: 1.842e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.8470411527341244
Training r2_score of Lasso() is  0.967597828022864


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.891e+10, tolerance: 1.832e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


...

C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.422e+10, tolerance: 1.911e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.854833614869895
Training r2_score of Lasso() is  0.9631112140602118


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.271e+10, tolerance: 1.813e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Testing r2_score of Lasso() is  0.9105528389652333
Training r2_score of Lasso() is  0.9581812091386825
Testing r2_score of Lasso() is  0.8882149970488236
Training r2_score of Lasso() is  0.9591348713692516


C:\Users\mritu\.conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.990e+10, tolerance: 1.793e+08
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [121]:
for i in range(50):

    algo_testing(algo="Ridge()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Ridge()"]/=50

Testing r2_score of Ridge() is  0.9145769488567369
Training r2_score of Ridge() is  0.9566355814530816
Testing r2_score of Ridge() is  0.8931522167842728
Training r2_score of Ridge() is  0.9604886848156298
Testing r2_score of Ridge() is  0.8962608050269083
Training r2_score of Ridge() is  0.9586583023280848
Testing r2_score of Ridge() is  0.9128470560424192
Training r2_score of Ridge() is  0.9595869277204337
Testing r2_score of Ridge() is  0.9181910742932216
Training r2_score of Ridge() is  0.9578646030991246
Testing r2_score of Ridge() is  0.9181465696347768
Training r2_score of Ridge() is  0.9559959576196421
Testing r2_score of Ridge() is  0.9250260639069638
Training r2_score of Ridge() is  0.9574825218494195
Testing r2_score of Ridge() is  0.8776505773268556
Training r2_score of Ridge() is  0.9622111967958337
Testing r2_score of Ridge() is  0.9063746562865603
Training r2_score of Ridge() is  0.9590672525262646
Testing r2_score of Ridge() is  0.9140803280897315
Training r2_score of R

In [122]:
avg_accuracies

{'LinearRegression()': 0.8792522809728812,
 'RandomForestRegressor()': 0.8692086221909165,
 'SVR()': -0.014744885602792395,
 'KNeighborsRegressor()': 0.6420365254601456,
 'SGDRegressor()': -5.862759653801886e+23,
 'DecisionTreeRegressor()': 0.7232604216241577,
 'ElasticNet()': 0.8859731091084613,
 'Lasso()': 0.8890567865280883,
 'Ridge()': 0.904646231461947}

In [123]:
highest_accuracies

{'LinearRegression()': 0.925638306335556,
 'RandomForestRegressor()': 0.9216759878163818,
 'SVR()': 0.0001268930354461073,
 'KNeighborsRegressor()': 0.7213272823237553,
 'SGDRegressor()': 0,
 'DecisionTreeRegressor()': 0.844324285786225,
 'ElasticNet()': 0.9292297130215331,
 'Lasso()': 0.9271873278356717,
 'Ridge()': 0.9376172994477122}